In [1]:
import numpy as np
import pandas as pd


In [2]:
dataset = pd.read_csv('../input/phishing-website-detector/phishing.csv')

In [3]:
dataset.head()

,Index,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,0,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,2,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,3,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,4,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11054 entries, 0 to 11053
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Index                11054 non-null  int64
 1   UsingIP              11054 non-null  int64
 2   LongURL              11054 non-null  int64
 3   ShortURL             11054 non-null  int64
 4   Symbol@              11054 non-null  int64
 5   Redirecting//        11054 non-null  int64
 6   PrefixSuffix-        11054 non-null  int64
 7   SubDomains           11054 non-null  int64
 8   HTTPS                11054 non-null  int64
 9   DomainRegLen         11054 non-null  int64
 10  Favicon              11054 non-null  int64
 11  NonStdPort           11054 non-null  int64
 12  HTTPSDomainURL       11054 non-null  int64
 13  RequestURL           11054 non-null  int64
 14  AnchorURL            11054 non-null  int64
 15  LinksInScriptTags    11054 non-null  int64
 16  ServerFormHandler    1

In [5]:
dataset.isnull().sum()

Index                  0
UsingIP                0
LongURL                0
ShortURL               0
Symbol@                0
Redirecting//          0
PrefixSuffix-          0
SubDomains             0
HTTPS                  0
DomainRegLen           0
Favicon                0
NonStdPort             0
HTTPSDomainURL         0
RequestURL             0
AnchorURL              0
LinksInScriptTags      0
ServerFormHandler      0
InfoEmail              0
AbnormalURL            0
WebsiteForwarding      0
StatusBarCust          0
DisableRightClick      0
UsingPopupWindow       0
IframeRedirection      0
AgeofDomain            0
DNSRecording           0
WebsiteTraffic         0
PageRank               0
GoogleIndex            0
LinksPointingToPage    0
StatsReport            0
class                  0
dtype: int64

In [6]:
Y = dataset["class"]
X = dataset.drop(["class"],axis=1)

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [8]:
Y

0       -1
1       -1
2       -1
3        1
4        1
        ..
11049    1
11050   -1
11051   -1
11052   -1
11053   -1
Name: class, Length: 11054, dtype: int64

## Training Using Different Models

In [9]:
#Using Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lg = LogisticRegression()
lg.fit(X_train,Y_train)
pred = lg.predict(X_test)
print(accuracy_score(pred,Y_test))

0.9167797376752601


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [10]:
#Using SVM 
from sklearn import svm
sv = svm.SVC()
sv.fit(X_train,Y_train)
pred_svm = sv.predict(X_test)

In [11]:
print(accuracy_score(pred_svm,Y_test))

0.5739484396200815


In [12]:
Y_train

5297     1
3420     1
8196    -1
7470     1
3858     1
        ..
4859     1
3264     1
9845     1
10799    1
2732     1
Name: class, Length: 8843, dtype: int64

In [13]:
y_train = Y_train.replace({1:1,-1:0})

In [14]:
print(y_train)

5297     1
3420     1
8196     0
7470     1
3858     1
        ..
4859     1
3264     1
9845     1
10799    1
2732     1
Name: class, Length: 8843, dtype: int64


In [15]:
y_test = Y_test.replace({1:1,-1:0})

In [16]:
#Using XGB 
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train,y_train)
pred_xg = xg.predict(X_test)
print(accuracy_score(pred_xg,y_test))

0.9651741293532339


## Printing the Classification Report and Confusion Matrix for XGB Classifier

In [17]:
from sklearn.metrics import classification_report,confusion_matrix
classification_repo = classification_report(pred_xg,y_test)

In [18]:
confusion_mat = confusion_matrix(pred_xg,y_test)

In [19]:
print(confusion_mat,"\n",classification_repo)

[[ 902   37]
 [  40 1232]] 
               precision    recall  f1-score   support

           0       0.96      0.96      0.96       939
           1       0.97      0.97      0.97      1272

    accuracy                           0.97      2211
   macro avg       0.96      0.96      0.96      2211
weighted avg       0.97      0.97      0.97      2211

